In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sqlmodel import create_engine, SQLModel, Session
from dotenv import load_dotenv

load_dotenv()

db_host = os.environ.get("DB_HOST")
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")
db_database = os.environ.get("DB_DATABASE")
db_port = os.environ.get("DB_PORT")


postgres_uri = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}"

engine = create_engine(postgres_uri, echo=False)

videosTable = "youtube_videos"
videoStatsTable = "youtube_video_stats"
channelsTable = "youtube_channels"
statsTable = "youtube_stats"

## Video scraping stats
- Number of videos at about page
- Number of videos scrapped
- Number of videos with stats


In [3]:
query = f'''
SELECT 
    {channelsTable}.*,
    MAX({statsTable}.videos) as about_videos,
    COUNT(DISTINCT {videosTable}.uuid) as n_videos,
    COUNT(DISTINCT {videoStatsTable}.uuid) as n_stats,
    COUNT(DISTINCT CASE WHEN {videoStatsTable}.uuid IS NULL THEN {videosTable}.uuid ELSE NULL END) as videos_no_stat,
    (MAX({statsTable}.videos) - COUNT(DISTINCT {videosTable}.uuid)) as not_found_videos
from {videosTable}
LEFT JOIN {channelsTable} ON {channelsTable}.uuid = {videosTable}.youtube_channel
LEFT JOIN {statsTable} ON {statsTable}.youtube_channel = {channelsTable}.uuid
LEFT JOIN {videoStatsTable} ON {videoStatsTable}.video = {videosTable}.uuid
GROUP BY
    {channelsTable}.uuid
'''
df = pd.read_sql(query, engine, index_col="uuid")
df

,username,about_videos,n_videos,n_stats,videos_no_stat,not_found_videos
uuid,,,,,,
1775cf1c-7f48-434b-89b7-6e6fde8c2dfb,ProgramadorX,164.0,146,146,0,18.0
67c82473-cf02-45c2-8022-834ba436e1ff,CarlosAzaustre,405.0,232,260,0,173.0
6e306e97-463f-4ba2-8d64-fa0c39f3965f,CodelyTV,554.0,353,443,0,201.0
7b0dc0ff-8d42-4e5f-94c0-536bf4a9d428,AMPTech,92.0,79,85,0,13.0
83d7f202-88b1-4761-a32a-fab990e18227,mouredev,507.0,335,335,0,172.0
844f783d-93e0-4451-8ea2-30054b3742f5,HolaMundoDev,334.0,260,260,0,74.0
867cc39e-785f-4ef9-80ae-368750523ad4,AntonioSarosi,49.0,47,47,0,2.0
d97bb78c-1c14-4bb3-a921-728222390e1f,BettaTech,167.0,153,153,0,14.0


## Channel stats
- Subs
- Time from start
- Views / total subs
- View / total videos
- Video with more views
- Video with more likes
- Video with more comments
- Avg views
- Avg likes
- Avg comments
- Subs per video

In [124]:
videoStatsQuery = f'''
SELECT
    {videoStatsTable}.video,
    MAX({videoStatsTable}.views) as views,
    MAX({videoStatsTable}.likes) as likes,
    MAX({videoStatsTable}.comments) as comments
FROM {videoStatsTable}
WHERE {videoStatsTable}.timestamp = (
    SELECT MAX(timestamp) FROM {videoStatsTable} AS latest
    WHERE latest.video = {videoStatsTable}.video
)
GROUP BY {videoStatsTable}.video
'''


channelStatsQuery = f'''
SELECT
    {statsTable}.youtube_channel,
    MAX({statsTable}.subs) as subs,
    MAX({statsTable}.videos) as about_videos
FROM {statsTable}
WHERE {statsTable}.timestamp = (
    SELECT MAX(timestamp) FROM {statsTable} AS latest
    WHERE latest.youtube_channel = {statsTable}.youtube_channel
)
GROUP BY {statsTable}.youtube_channel
'''




query = f'''
SELECT 
    {channelsTable}.username,
    COUNT({videosTable}.uuid) as videos,
    AVG(channel_stats.about_videos) as about_videos,
    AVG(channel_stats.subs) as subs,
    SUM(latest_video_stats.views) as views,
    SUM(latest_video_stats.likes) as likes,
    SUM(latest_video_stats.comments) as comments,
    AVG(latest_video_stats.views) as avg_views,
    AVG(latest_video_stats.likes) as avg_likes,
    AVG(latest_video_stats.comments) as avg_comments,
    MIN(latest_video_stats.views) as min_views,
    MIN(latest_video_stats.likes) as min_likes,
    MIN(latest_video_stats.comments) as min_comments,
    (SUM(latest_video_stats.views)/AVG(channel_stats.subs)) as views_per_sub,
    (SUM(latest_video_stats.likes) / SUM(latest_video_stats.views)) as likes_per_views,
    (SUM(latest_video_stats.comments) / SUM(latest_video_stats.views)) as comments_per_views,
    (AVG(channel_stats.subs) / COUNT({videosTable}.uuid)) as subs_per_video
FROM {videosTable}
    LEFT JOIN {channelsTable} ON {channelsTable}.uuid = {videosTable}.youtube_channel
    LEFT JOIN (
        {videoStatsQuery}
    ) latest_video_stats ON latest_video_stats.video = {videosTable}.uuid
    LEFT JOIN (
        {channelStatsQuery}
    ) channel_stats ON channel_stats.youtube_channel = {videosTable}.youtube_channel
GROUP BY {channelsTable}.username
'''


df = pd.read_sql(query, engine)
df

,username,videos,about_videos,subs,views,likes,comments,avg_views,avg_likes,avg_comments,min_views,min_likes,min_comments,views_per_sub,likes_per_views,comments_per_views,subs_per_video
0,AMPTech,79,92.0,50200.0,2152875.0,71221.0,2135.0,27251.582278,901.531646,27.025316,0.0,0.0,0.0,42.885956,0.033082,0.000992,635.443038
1,AntonioSarosi,47,49.0,166000.0,7090567.0,187536.0,7516.0,150863.127660,3990.127660,159.914894,0.0,0.0,0.0,42.714259,0.026449,0.001060,3531.914894
2,BettaTech,153,167.0,226000.0,9966405.0,421024.0,25671.0,65139.901961,2751.790850,167.784314,1835.0,0.0,0.0,44.099137,0.042244,0.002576,1477.124183
3,CarlosAzaustre,232,405.0,131000.0,6295731.0,206080.0,9003.0,27136.771552,888.275862,38.806034,0.0,4.0,0.0,48.059015,0.032733,0.001430,564.655172
4,CodelyTV,353,554.0,139000.0,5754340.0,218798.0,8006.0,16301.246459,619.824363,22.679887,0.0,0.0,0.0,41.398129,0.038023,0.001391,393.767705
5,HolaMundoDev,260,334.0,672000.0,31902038.0,595327.0,90363.0,122700.146154,2289.719231,347.550000,0.0,0.0,0.0,47.473271,0.018661,0.002833,2584.615385
6,mouredev,335,507.0,407000.0,14916776.0,370783.0,12647.0,44527.689552,1106.814925,37.752239,0.0,0.0,0.0,36.650555,0.024857,0.000848,1214.925373
7,ProgramadorX,146,164.0,291000.0,10140648.0,397490.0,21845.0,69456.493151,2722.534247,149.623288,0.0,0.0,0.0,34.847588,0.039198,0.002154,1993.150685


## Videos stats

In [125]:
query = f'''
SELECT
    {videosTable}.title,
    MAX({channelsTable}.username) as channel,
    MAX({videoStatsTable}.views) as views,
    MAX({videoStatsTable}.likes) as likes,
    MAX({videoStatsTable}.comments) as comments,
    MAX({videoStatsTable}.date) as date,
    CASE
        WHEN MAX({statsTable}.subs) > 0 THEN MAX({videoStatsTable}.views) / MAX({statsTable}.subs)
        ELSE 0.0 -- Handle division by zero by returning 0.0 (you can use NULL or another value as well)
    END as views_per_subs,
    CASE
        WHEN MAX({videoStatsTable}.views) > 0 THEN MAX({videoStatsTable}.likes) / MAX({videoStatsTable}.views)
        ELSE 0.0 -- Handle division by zero by returning 0.0 (you can use NULL or another value as well)
    END as likes_per_views,
    CASE
        WHEN MAX({videoStatsTable}.views) > 0 THEN MAX({videoStatsTable}.comments) / MAX({videoStatsTable}.views)
        ELSE 0.0 -- Handle division by zero by returning 0.0 (you can use NULL or another value as well)
    END as comments_per_views,
    MAX({statsTable}.subs) as subs
FROM {videosTable}
    LEFT JOIN {videoStatsTable} ON {videoStatsTable}.video = {videosTable}.uuid
    LEFT JOIN {statsTable} ON {statsTable}.youtube_channel = {videosTable}.youtube_channel
    LEFT JOIN {channelsTable} ON {channelsTable}.uuid = {videosTable}.youtube_channel
GROUP BY {videosTable}.title
'''
df = pd.read_sql(query, engine, index_col='title')
df


,channel,views,likes,comments,date,views_per_subs,likes_per_views,comments_per_views,subs
title,,,,,,,,,
4 Formas de crear un Component en React,CarlosAzaustre,15000.0,272.0,0.0,None,0.114504,0.018133,0.000000,131000.0
"La caída de las cryptos, cómo ganar dinero en crypto invierno?",HolaMundoDev,20432.0,1576.0,128.0,2022-05-27,0.030405,0.077134,0.006265,672000.0
Experiencia como Product Designer en NewRelic y workshop ReactJS en #iwdbcn18 | Dulce D' La Rosa,CodelyTV,344.0,10.0,0.0,2018-04-18,0.002475,0.029070,0.000000,139000.0
"Python sin fronteras, curso de programación desde cero",HolaMundoDev,140136.0,0.0,884.0,2020-10-07,0.208536,0.000000,0.006308,672000.0
que es un desarrollador fullstack,HolaMundoDev,96791.0,7414.0,236.0,2019-05-26,0.144034,0.076598,0.002438,672000.0
...,...,...,...,...,...,...,...,...,...
Cómo crear tu WEB con NOTION | Con Miriam Gonzalez 🔴 HackerLIVE#32,mouredev,0.0,304.0,0.0,None,0.000000,0.000000,0.000000,407000.0
Hackeé una startup y sus usuarios | fue demasiado fácil!,HolaMundoDev,115979.0,8771.0,223.0,2022-06-10,0.172588,0.075626,0.001923,672000.0
5 consejos para SUPERAR el SÍNDROME del IMPOSTOR,BettaTech,33998.0,2469.0,126.0,2021-01-15,0.150434,0.072622,0.003706,226000.0


In [127]:
totalVideos = len(df.index)
nullDates = np.sum(pd.isnull(df['date']))
print(f"Null dates: {nullDates}/{totalVideos} ({round(nullDates/totalVideos*100, 2)}%)")

nullViews = np.sum(pd.isnull(df['views']))
print(f"Null views: {nullViews}/{totalVideos} ({round(nullViews/totalVideos*100, 2)}%)")

nullSubs = np.sum(pd.isnull(df['subs']))
print(f"Null subs: {nullSubs}/{totalVideos} ({round(nullSubs/totalVideos*100, 2)}%)")

df.sort_values(by=["views"], ascending=False).head(30)

Null dates: 691/1605 (43.05%)
Null views: 0/1605 (0.0%)
Null subs: 0/1605 (0.0%)


,channel,views,likes,comments,date,views_per_subs,likes_per_views,comments_per_views,subs
title,,,,,,,,,
KOTLIN: Curso ANDROID desde CERO para PRINCIPIANTES,mouredev,2952652.0,0.0,1365.0,2020-03-23,7.254673,0.000000,0.000462,407000.0
ANDROID STUDIO: COMO Crear una APP (para Principiantes) 📲 [Tutorial],mouredev,2300000.0,0.0,0.0,None,5.651106,0.000000,0.000000,407000.0
Lógica de Programación 👩‍💻 Aprende a programar en 10 minutos,ProgramadorX,1977477.0,0.0,2340.0,2021-03-12,6.795454,0.000000,0.001183,291000.0
Aprende HTML ahora! curso completo GRATIS desde cero,HolaMundoDev,1700000.0,0.0,0.0,None,2.529762,0.000000,0.000000,672000.0
Los programadores mejor pagados del mundo,HolaMundoDev,1128241.0,0.0,2532.0,2020-07-17,1.678930,0.000000,0.002244,672000.0
Curso de PYTHON desde CERO para PRINCIPIANTES,mouredev,1100000.0,0.0,0.0,None,2.702703,0.000000,0.000000,407000.0
Esto es lo que te encontrarás en el primer año de Ingeniería Informática,AntonioSarosi,966000.0,0.0,0.0,None,5.819277,0.000000,0.000000,166000.0
Aprende CSS ahora! curso completo GRATIS desde cero,HolaMundoDev,901000.0,0.0,0.0,None,1.340774,0.000000,0.000000,672000.0
Pasando de Noob a Pro de Linux en 20 Minutos,AntonioSarosi,880701.0,0.0,1708.0,2020-03-27,5.305428,0.000000,0.001939,166000.0


## Export to JSON

In [91]:
df.to_json("dataframe.json")

### Seed Questions
 - ¿Which is the best topic to make videos?
 - ¿Who is the best comunicator?
 - ¿Which is the most viewed video?